## Import and Upload the Data into Exasol

We are going to import a csv file that we downlaod from Kaggle that is a dataset of a store data

### Open Configuration

In [1]:
%run ../utils/access_store_ui.ipynb
display(get_access_store_ui('../'))


Output()

Box(children=(Box(children=(Label(value='Configuration Store', layout=Layout(border_bottom='solid 1px', border…

### Import dataset into pandas dataframe

From the csv file. We are importing the data into the Pandas Dataframe and converting the 'Order Date' and 'Ship Date' column to the pandas datetime format because that is the column on which we are going to predict on.

In [2]:
from pandas import pandas as pd
from sklearn.preprocessing import LabelEncoder
data = pd.read_csv('stores_sales_forecasting.csv', encoding='ISO-8859-1')
data['Order Date'] = pd.to_datetime(data['Order Date'],errors='coerce')
data['Ship Date'] = pd.to_datetime(data['Ship Date'], errors='coerce')
data.head()

Row ID        Order ID Order Date  Ship Date       Ship Mode Customer ID  \
0       1  CA-2016-152156 2016-11-08 2016-11-11    Second Class    CG-12520   
1       2  CA-2016-152156 2016-11-08 2016-11-11    Second Class    CG-12520   
2       4  US-2015-108966 2015-10-11 2015-10-18  Standard Class    SO-20335   
3       6  CA-2014-115812 2014-06-09 2014-06-14  Standard Class    BH-11710   
4      11  CA-2014-115812 2014-06-09 2014-06-14  Standard Class    BH-11710   

     Customer Name   Segment        Country             City  ... Postal Code  \
0      Claire Gute  Consumer  United States        Henderson  ...       42420   
1      Claire Gute  Consumer  United States        Henderson  ...       42420   
2   Sean O'Donnell  Consumer  United States  Fort Lauderdale  ...       33311   
3  Brosina Hoffman  Consumer  United States      Los Angeles  ...       90032   
4  Brosina Hoffman  Consumer  United States      Los Angeles  ...       90032   

   Region       Product ID   Category Sub-Category  \
0   South  FUR-BO-10001798  Furniture    Bookcases   
1   South  FUR-CH-10000454  Furniture       Chairs   
2   South  FUR-TA-10000577  Furniture       Tables   
3    West  FUR-FU-10001487  Furniture  Furnishings   
4    West  FUR-TA-10001539  Furniture       Tables   

                                        Product Name      Sales  Quantity  \
0                  Bush Somerset Collection Bookcase   261.9600         2   
1  Hon Deluxe Fabric Upholstered Stacking Chairs,...   731.9400         3   
2      Bretford CR4500 Series Slim Rectangular Table   957.5775         5   
3  Eldon Expressions Wood and Plastic Desk Access...    48.8600         7   
4           Chromcraft Rectangular Conference Tables  1706.1840         9   

   Discount    Profit  
0      0.00   41.9136  
1      0.00  219.5820  
2      0.45 -383.0310  
3      0.00   14.1694  
4      0.20   85.3092  

[5 rows x 21 columns]

### Creating features for our XGBoost Model

The "create_features" function generates new features for a sales dataset, including time-based features (day, month, year, weekday) from 'Order Date' and 'Ship Date'. It creates lag features for sales (1-week and 1-month), calculates a 'quantity_discount_ratio', and one-hot encodes categorical variables. The function then removes irrelevant columns and handles missing values, returning the processed dataset for further analysis or modeling.

In [3]:

def create_features_new(data,lags=[1,3,7,30]):
    """
    Creates time series features from datetime index
    """
    data['order_day'] = data['Order Date'].dt.day
    data['sales'] = data['Sales']
    data['order_month'] = data['Order Date'].dt.month
    data['order_year'] = data['Order Date'].dt.year
    data['order_weekday'] = data['Order Date'].dt.weekday
    data['ship_day'] = data['Ship Date'].dt.day
    data['ship_month'] = data['Ship Date'].dt.month
    data['ship_year'] = data['Ship Date'].dt.year
    data['ship_weekday'] = data['Ship Date'].dt.weekday
    categorical_columns = ['Segment', 'Country', 'City', 'Region', 'Category', 'Sub-Category', 'Ship Mode','State']
    data_encoded = pd.get_dummies(data, columns=categorical_columns, drop_first=True)
    data_encoded['sales_lag_1_week'] = data_encoded['Sales'].shift(7)
    data_encoded['sales_lag_1_month'] = data_encoded['Sales'].shift(30)
    data_encoded['quantity_discount_ratio'] = data_encoded['Quantity'] / (data_encoded['Discount'] + 1)
    data_encoded.dropna()
    
  
    return data_encoded, data_encoded.drop(columns=['Sales', 'Row ID', 'Order ID', 'Customer ID', 'Customer Name', 'Product Name', 'Product ID', 'Ship Date', 'Order Date'])

#### Executing the function

In [5]:

train_features, test =  create_features_new(data)

### Create a table in the database and load the data into that table

In [6]:
from exasol.nb_connector.connections import open_pyexasol_connection

dtype_map = {
    'int64': 'INTEGER',
    'float64': 'FLOAT',
    'object': 'VARCHAR(255)',
    'bool': 'BOOLEAN',
    'datetime64[ns]': 'TIMESTAMP'
}

train_table = "TRAIN_TABLE"
test_table = "TEST_TABLE"
# Generating a CREATE TABLE statement
cols = []
for col_name, dtype in train_features.dtypes.items():
    sql_type = dtype_map.get(str(dtype), 'VARCHAR(255)')
    cols.append(f'"{col_name}" {sql_type}')

test_cols = []
for col_name, dtype in test.dtypes.items():
    sql_type = dtype_map.get(str(dtype), 'VARCHAR(255)')
    test_cols.append(f'"{col_name}" {sql_type}')
    
with open_pyexasol_connection(ai_lab_config, compression=True) as conn:
    create_train_sql = f'CREATE OR REPLACE TABLE "{ai_lab_config.db_schema}"."{train_table}"({", ".join(cols)})'
    conn.execute(create_train_sql)
    create_test_sql = f'CREATE OR REPLACE TABLE "{ai_lab_config.db_schema}"."{test_table}"({", ".join(test_cols)})'
    conn.execute(create_test_sql)

    conn.import_from_pandas(train_features, (ai_lab_config.db_schema, train_table))
    print(f"Imported {conn.last_statement().rowcount()} rows into '{train_table}' table")
    conn.import_from_pandas(test, (ai_lab_config.db_schema, test_table))
   
    print(f"Imported {conn.last_statement().rowcount()} rows into '{test_table}' table")
    

Imported 2121 rows into 'TRAIN_TABLE' table
Imported 2121 rows into 'TEST_TABLE' table


Now we can move onto [creating and uploading a UDF into the database](./1-create_udf.ipynb) to execute the predict function on our test dataset.